In [3]:
from keras import applications
import keras
import tensorflow as tf

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint,CSVLogger
from keras import backend as k



DATASET_PATH  = 'Cleaned'
IMAGE_SIZE    = (200,200)                                                                                                                                           
NUM_CLASSES   = 2
BATCH_SIZE    = 32  # try reducing batch size or freeze more layers if your GPU runs out of memory                                                                                                                                                                             
NUM_EPOCHS    = 30
WEIGHTS_FINAL = 'MobileNetV2model-transfer-Chest-MobileNet-000001--final.h5'


In [6]:

train_datagen = ImageDataGenerator( rescale=1.0 / 255.0,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                    channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/val',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_batches = test_datagen.flow_from_directory(DATASET_PATH + '/test',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)
lrelu = lambda x: tensorflow.keras.activations.relu(x, alpha=0.1)


Found 2053 images belonging to 2 classes.
Found 383 images belonging to 2 classes.
Found 130 images belonging to 2 classes.


In [4]:


# Load VGG16 model architecture with the ImageNet weights                                                                                                                                        
model = tf.keras.applications.MobileNetV2(weights = "imagenet", include_top=False, input_shape=[200,200,3])


# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.                                               
for layer in model.layers:
    layer.trainable = False


# Build classifier
x = model.output
x = Flatten()(x)
x = Dense(32, activation="sigmoid")(x)
predictions = Dense(2, activation="softmax")(x)


#Use Adam optimizer (instead of plain SGD), set learning rate to explore.                                                                                                                                                                                                      
adam = Adam(learning_rate=.00001)

#instantiate model                                                                                                                                                                                                                                                             
model = tf.keras.Model(inputs=model.input, outputs=predictions)


9420800/9406464 [==============================] - 0s 0us/step


In [5]:

#Compile model                                                                                                                                                                                                                                                                 
model.compile(optimizer = adam, loss='categorical_crossentropy', metrics=['accuracy'])

#Print layers for resulting model                                                                                                                                                                                                                                              
model.summary()
#Tim: MobileNetV2, ResNet
#Log training data into csv file                                                                                                                                                                                                                                               
csv_logger = CSVLogger(filename="MobileNetV2-imagenet-log.csv")
# checkpointer = ModelCheckpoint(filepath='VGG16/weights.{epoch:02d}-{val_acc:.2f}.hdf5',monitor='val_loss', verbose=1, save_best_only=True, mode='min')
cblist = [csv_logger]

# train the model                                                                                                                                                                                                                                                              
model.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
            callbacks=cblist)

# save trained model and weights                                                                                                                                                                                                                                               
model.save(WEIGHTS_FINAL)



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 100, 100, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 100, 100, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 100, 100, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

/home/inzi/.local/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
64/64 [==============================] - 19s 264ms/step - loss: 0.5976 - accuracy: 0.6833 - val_loss: 0.5346 - val_accuracy: 0.7216
Epoch 2/30
64/64 [==============================] - 17s 259ms/step - loss: 0.5044 - accuracy: 0.7531 - val_loss: 0.4843 - val_accuracy: 0.7557
Epoch 3/30
64/64 [==============================] - 16s 257ms/step - loss: 0.4584 - accuracy: 0.7917 - val_loss: 0.4585 - val_accuracy: 0.7614
Epoch 4/30
64/64 [==============================] - 16s 253ms/step - loss: 0.4374 - accuracy: 0.8026 - val_loss: 0.4396 - val_accuracy: 0.7784
Epoch 5/30
64/64 [==============================] - 16s 255ms/step - loss: 0.4119 - accuracy: 0.8164 - val_loss: 0.4259 - val_accuracy: 0.7983
Epoch 6/30
64/64 [==============================] - 16s 253ms/step - loss: 0.4054 - accuracy: 0.8234 - val_loss: 0.4169 - val_accuracy: 0.8011
Epoch 7/30
64/64 [==============================] - 16s 256ms/step - loss: 0.3824 - accuracy: 0.8362 - val_loss: 0.4006 - val_accuracy: 0.8040

/home/inzi/.local/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [6]:
model.predict(test_batches)

array([[0.9816232 , 0.01837682],
       [0.9788263 , 0.02117368],
       [0.96525276, 0.03474722],
       [0.86145914, 0.13854088],
       [0.94752204, 0.05247794],
       [0.65912944, 0.34087056],
       [0.72822684, 0.27177316],
       [0.5211276 , 0.47887242],
       [0.6901181 , 0.30988196],
       [0.7000204 , 0.29997966],
       [0.6852485 , 0.31475154],
       [0.38115758, 0.6188424 ],
       [0.38067216, 0.6193279 ],
       [0.4014369 , 0.5985631 ],
       [0.9450236 , 0.05497633],
       [0.9458133 , 0.0541867 ],
       [0.9453234 , 0.05467664],
       [0.8663354 , 0.13366464],
       [0.70522135, 0.29477862],
       [0.9442506 , 0.05574937],
       [0.8656456 , 0.13435435],
       [0.8495731 , 0.15042691],
       [0.74168396, 0.258316  ],
       [0.88189393, 0.118106  ],
       [0.45654872, 0.5434513 ],
       [0.95822465, 0.04177539],
       [0.21198365, 0.7880163 ],
       [0.70055336, 0.2994466 ],
       [0.18743643, 0.81256354],
       [0.8332444 , 0.16675563],
       [0.

In [7]:
y = model.predict(test_batches)

In [8]:
actual_class = test_batches.classes
print(actual_class)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [9]:
total = 0
total_correct = 0
for i in range(0, len(actual_class)):
  cat0 = y[i][0]
  cat1 = y[i][1]
  if cat0 > cat1:
    if actual_class[i] == 0:
      total_correct = total_correct + 1
  else: 
    if actual_class[i] == 1:
      total_correct = total_correct + 1
  total = total + 1

In [10]:
print(f"total: {total}")
print(f"total correct: {total_correct}")
print(f"average: {total_correct/total}")

total: 130
total correct: 112
average: 0.8615384615384616
